# Explainable AI: COMPAS Predictions

Importing libraries

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

Importing data

In [ ]:
# change file name for data if using different version
dfOriginal = pd.read_csv("cox-violent-parsed_filt.csv")

Remove duplicates, only one row per name

In [ ]:
dfProcessed = dfOriginal.drop_duplicates(subset=['name'])
dfProcessed.count()

Remove unused columns

In [ ]:
dfProcessed = dfProcessed[['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree', 'is_recid']]
dfProcessed.count()

Remove -1 is_recid (must be binary)

In [ ]:
# What we expect to be dropped
# Just using this to double check
dfCheck = dfProcessed.loc[dfProcessed['is_recid'] < 0]
dfCheck.count()

In [ ]:
# Dropping the invalid is_recid values
dfProcessed = dfProcessed.loc[dfProcessed['is_recid'] > -1]
dfProcessed.count()

Missing value strategy
1. Numerical values --> MEDIAN imputation
2. Categorical values --> MODE imputation

In [ ]:
print("Any NaN values?\n", dfProcessed.isnull().values.any())
# ^^ To check if any NaNs
# put it bc I tried a heatmap and it looked empty so double checking

In [ ]:
# Numerical columns
numCols = ['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count']
numImputer = SimpleImputer(strategy = 'median')
dfProcessed[numCols] = numImputer.fit_transform(dfProcessed[numCols])

# Categorical columns
catCols = ['sex', 'race', 'c_charge_degree']
catImputer = SimpleImputer(strategy = 'most_frequent') # most_frequent = mode
dfProcessed[catCols] = catImputer.fit_transform(dfProcessed[catCols])

print("Any NaN values?\n", dfProcessed.isnull().values.any())
# no more NaNs :)

Smote? Impute from other dataset? Impute from same dataset?

In [ ]:
# We should discuss what the plan is for this part

Dummy Model
(idk how to approach this)

In [ ]:
# Not sure how to approach the dummy model

Random Forest

In [ ]:
target = dfProcessed['is_recid'] # x-axis
features = dfProcessed.drop('is_recid', axis = 1) # y-axis

testSize = 0.3 # change this variable if you want different train/test split
X_train, X_test, y_train, y_test = train_test_split(target, features, test_size=testSize, random_state=44)